This pipeline designed for verifying rotation and alignment in 2D eye data and rotation matrices acquired from tear ducts - use before going into kerr degree conversion

In [106]:

import numpy as np
import cv2

import pickle
import pathlib

import pandas as pd

from BlockSync_current import BlockSync


In [123]:
def load_eye_data_2d_w_rotation_matrix(block):
    """
    This function checks if the eye dataframes and rotation dict object exist, then imports them
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    try:
        block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except FileNotFoundError:
        print('eye_data files not found, run the pipeline!')
        return

    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as file:
            rotation_dict = pickle.load(file)
            block.left_rotation_matrix = rotation_dict['left_rotation_matrix']
            block.right_rotation_matrix = rotation_dict['right_rotation_matrix']
            block.left_rotation_angle = rotation_dict['left_rotation_angle']
            block.right_rotation_angle = rotation_dict['right_rotation_angle']
    except FileNotFoundError:
        print('No rotation matrix file, create it')


# Interactive verification/correction version over here:
from pathlib import Path


def horizontal_flip_eye_data(df: pd.DataFrame, frame_width: int) -> pd.DataFrame:
    df2 = df.copy()
    df2['center_x'] = frame_width - df2['center_x']
    df2['phi'] = (180 - df2['phi']) % 360
    return df2


def apply_inverse_rotation(df: pd.DataFrame, rot_mat: np.ndarray, rot_angle: float) -> pd.DataFrame:
    inv = cv2.invertAffineTransform(rot_mat.astype(np.float32))
    df2 = df.copy()
    pts = df2[['center_x', 'center_y']].values.reshape(-1, 1, 2).astype(np.float32)
    pts2 = cv2.transform(pts, inv)
    df2['center_x'] = pts2[:, 0, 0]
    df2['center_y'] = pts2[:, 0, 1]
    df2['phi'] = (df2['phi'] - rot_angle) % 360
    return df2


def apply_rotation(df: pd.DataFrame, rot_mat: np.ndarray, rot_angle: float) -> pd.DataFrame:
    df2 = df.copy()
    pts = df2[['center_x', 'center_y']].values.reshape(-1, 1, 2).astype(np.float32)
    pts2 = cv2.transform(pts, rot_mat.astype(np.float32))
    df2['center_x'] = pts2[:, 0, 0]
    df2['center_y'] = pts2[:, 0, 1]
    df2['phi'] = (df2['phi'] + rot_angle) % 360
    return df2


def rotate_phi_only(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    df2['phi'] = (df2['phi'] + 90) % 360
    return df2


def flip_x_only(df: pd.DataFrame, frame_width: int) -> pd.DataFrame:
    df2 = df.copy()
    df2['center_x'] = frame_width - df2['center_x']
    return df2


def interactive_eye_data_corrector_synced(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor with Play/Pause, correction, Save,
    Flip-Dot, and Skip-forward/backward (1 minute) buttons.

    Parameters
    ----------
    block : BlockSync
        Your BlockSync instance with loaded eye_data and rotation_matrix attributes.
    eye : str
        'left' or 'right'
    ref_point_xy : tuple[int,int] or None
        If provided, a (x,y) coordinate in raw frame space to draw as a blue dot on every frame.
    """
    import cv2
    import numpy as np
    import pandas as pd

    # 1) select data & video
    if eye.lower() == 'left':
        df_orig = block.left_eye_data.copy()
        rot_mat = np.array(block.left_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.left_rotation_angle)
        video = block.le_videos[0]
    else:
        df_orig = block.right_eye_data.copy()
        rot_mat = np.array(block.right_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.right_rotation_angle)
        video = block.re_videos[0]

    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip_frames = int(fps * 60)  # skip 1 minute

    # 2) prepare DataFrame & frame index column
    df_current = df_orig.copy()
    frame_col = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # 3) define buttons & layout
    buttons = {
        'Play': ((10, 10), (180, 60)),
        'Pause': ((10, 80), (180, 130)),
        'Un-rotate': ((10, 150), (180, 200)),
        'X-flip': ((10, 220), (180, 270)),
        'Re-rotate': ((10, 290), (180, 340)),
        'Phi+90': ((10, 360), (180, 410)),
        'FlipX-only': ((10, 430), (180, 480)),
        'Flip Dot': ((10, 500), (180, 550)),
        'Bwd': ((10, 570), (180, 620)),
        'Fwd': ((10, 640), (180, 690)),
        'Save': ((10, 710), (180, 760)),
        'Quit': ((10, 780), (180, 830)),
    }
    ctrl_h, ctrl_w = 860, 200

    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            cv2.rectangle(img, (x1, y1), (x2, y2), (50, 50, 50), -1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (200, 200, 200), 2)
            cv2.putText(img, name, (x1 + 5, y1 + 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2, cv2.LINE_AA)
        return img

    controls_img = draw_controls()
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)

    # 4) interaction state
    running = True
    playing = False
    current_ref = ref_point_xy
    last_frame = None

    # 5) mouse callback
    def on_mouse(event, x, y, flags, param):
        nonlocal df_current, running, playing, current_ref, last_frame
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                if name == 'Play':
                    playing = True
                elif name == 'Pause':
                    playing = False
                elif name == 'Un-rotate':
                    df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name == 'X-flip':
                    df_current = horizontal_flip_eye_data(df_current, W)
                elif name == 'Re-rotate':
                    df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name == 'Phi+90':
                    df_current = rotate_phi_only(df_current)
                elif name == 'FlipX-only':
                    df_current = flip_x_only(df_current, W)
                elif name == 'Flip Dot' and current_ref is not None:
                    x0, y0 = current_ref
                    current_ref = (W - x0, y0)
                elif name == 'Bwd':
                    # skip backward 1 minute
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    new_idx = max(idx - skip_frames, 0)
                    cap.set(cv2.CAP_PROP_POS_FRAMES, new_idx)
                    last_frame = None
                elif name == 'Fwd':
                    # skip forward 1 minute
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    new_idx = min(idx + skip_frames, total_frames - 1)
                    cap.set(cv2.CAP_PROP_POS_FRAMES, new_idx)
                    last_frame = None
                elif name == 'Save':
                    if eye.lower() == 'left':
                        block.left_eye_data = df_current.copy()
                    else:
                        block.right_eye_data = df_current.copy()
                    print(f"{eye.capitalize()} eye data saved.")
                elif name == 'Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # 6) play/pause loop
    while running:
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret:
                break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        # sync: get current frame index
        current_idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        current_idx = max(current_idx, 0)

        # draw reference dot if provided
        annotated = frame.copy()
        if current_ref is not None:
            cv2.circle(annotated, current_ref, 5, (255, 0, 0), -1)

        # draw ellipse if valid data exists
        mask = df_current[frame_col] == current_idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx, cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                x = int(round(cx))
                y = int(round(cy))
                w = int(row['width'])
                h = int(row['height'])
                phi = float(row['phi'])
                cv2.ellipse(annotated, (x, y), (w, h), phi, 0, 360, (0, 255, 0), 2)

        # final vertical flip for display
        disp = cv2.flip(annotated, 0)
        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        if cv2.waitKey(30) & 0xFF == 27:  # ESC to exit
            break

    cap.release()
    cv2.destroyAllWindows()


def interactive_eye_rotation_checker(block, eye):
    """
    Interactive tool to visualize how a rotation matrix would affect your
    un-rotated eye data and frame, by drawing the raw ellipse first and then
    warping the entire annotated frame.

    Buttons:
      • Play              : start auto-play
      • Pause             : stop auto-play
      • Original rotation : warp with block.<eye>_rotation_matrix
      • Reversed rotation : warp with inverse matrix
      • Quit              : exit

    Workflow per frame:
      1. Read raw frame (no flips).
      2. Draw ellipse at raw (center_x, center_y) with raw φ from DataFrame.
      3. Warp the *entire* annotated frame by the selected 2×3 matrix.
      4. Vertically flip for display.
    """
    import cv2
    import numpy as np
    import pandas as pd

    # 1) Pick eye‐specific data & matrices
    if eye.lower() == 'left':
        df = block.left_eye_data.copy()
        R_orig = np.array(block.left_rotation_matrix, dtype=np.float32)
        video = block.le_videos[0]
    else:
        df = block.right_eye_data.copy()
        R_orig = np.array(block.right_rotation_matrix, dtype=np.float32)
        video = block.re_videos[0]

    # Compute inverse rotation matrix
    R_rev = cv2.invertAffineTransform(R_orig)

    # Open video
    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Determine which column holds frame indices
    frame_col = 'eye_frame' if 'eye_frame' in df.columns else 'frame'

    # 2) Define buttons and layout
    buttons = {
        'Play': ((10, 10), (180, 60)),
        'Pause': ((10, 80), (180, 130)),
        'Original rotation': ((10, 150), (180, 200)),
        'Reversed rotation': ((10, 220), (180, 270)),
        'Quit': ((10, 290), (180, 340)),
    }
    ctrl_h, ctrl_w = 360, 200

    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            cv2.rectangle(img, (x1, y1), (x2, y2), (50, 50, 50), -1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (200, 200, 200), 2)
            cv2.putText(img, name, (x1 + 5, y1 + 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2, cv2.LINE_AA)
        return img

    controls_img = draw_controls()
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)

    # 3) Interaction state
    running = True
    playing = False
    use_matrix = R_orig  # start with original rotation

    def on_mouse(event, x, y, flags, param):
        nonlocal running, playing, use_matrix
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                if name == 'Play':
                    playing = True
                elif name == 'Pause':
                    playing = False
                elif name == 'Original rotation':
                    use_matrix = R_orig
                elif name == 'Reversed rotation':
                    use_matrix = R_rev
                elif name == 'Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # 4) Playback loop
    last_frame = None
    while running:
        # Advance if playing
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret:
                break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        # Sync: current frame index
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        if idx < 0: idx = 0

        # 5) Draw raw ellipse on native frame
        annotated = frame.copy()
        mask = df[frame_col] == idx
        if mask.any():
            row = df[mask].iloc[0]
            cx, cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                x = int(round(cx))
                y = int(round(cy))
                w = int(row['width'])
                h = int(row['height'])
                phi = float(row['phi'])
                cv2.ellipse(annotated, (x, y), (w, h), phi, 0, 360, (0, 255, 0), 2)

        # 6) Warp the entire annotated frame
        warped = cv2.warpAffine(
            annotated,
            use_matrix,
            (W, H),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)
        )

        # 7) Final vertical flip for display
        disp = cv2.flip(warped, 0)
        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        # 8) Exit on ESC
        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


def negate_eye_rotation(block, eye):
    """
    Replace block.<eye>_rotation_matrix with its inverse (negated rotation)
    and update block.<eye>_rotation_angle to -angle (mod 360).

    Parameters
    ----------
    block : your BlockSync instance
    eye : str
        'left' or 'right'
    """
    if eye.lower() == 'left':
        R_old = np.array(block.left_rotation_matrix, dtype=np.float32)
        ang_old = float(block.left_rotation_angle)
        invR = cv2.invertAffineTransform(R_old)
        block.left_rotation_matrix = invR
        block.left_rotation_angle = (-ang_old) % 360.0
        print(f"Left rotation matrix negated; angle set to {block.left_rotation_angle:.2f}°")
    elif eye.lower() == 'right':
        R_old = np.array(block.right_rotation_matrix, dtype=np.float32)
        ang_old = float(block.right_rotation_angle)
        invR = cv2.invertAffineTransform(R_old)
        block.right_rotation_matrix = invR
        block.right_rotation_angle = (-ang_old) % 360.0
        print(f"Right rotation matrix negated; angle set to {block.right_rotation_angle:.2f}°")
    else:
        raise ValueError("eye must be 'left' or 'right'")


def export_corrected_eye_data(block):
    """
    Overwrite the eye‐data CSVs and rotation‐params pickle so that
    load_eye_data_2d_w_rotation_matrix(block) will load the current,
    corrected attributes from disk.

    Writes:
      • block.analysis_path/'left_eye_data.csv'
      • block.analysis_path/'right_eye_data.csv'
      • block.analysis_path/'rotate_eye_data_params.pkl'
    """
    # Ensure the analysis_path exists
    analysis_path = Path(block.analysis_path)
    analysis_path.mkdir(parents=True, exist_ok=True)

    # 1) Write the DataFrames
    block.left_eye_data.to_csv(analysis_path / 'left_eye_data.csv', index=True)
    block.right_eye_data.to_csv(analysis_path / 'right_eye_data.csv', index=True)

    # 2) Build and write the rotation‐params pickle
    rot_dict = {
        'left_rotation_matrix': block.left_rotation_matrix,
        'left_rotation_angle': block.left_rotation_angle,
        'right_rotation_matrix': block.right_rotation_matrix,
        'right_rotation_angle': block.right_rotation_angle
    }
    with open(analysis_path / 'rotate_eye_data_params.pkl', 'wb') as f:
        pickle.dump(rot_dict, f)

    print(f"Exported corrected eye data and rotation params to {analysis_path}")




In [126]:
def create_block_collections(animals, block_lists, experiment_path, bad_blocks=None):
    """
    Create block collections and a block dictionary from multiple animals and their respective block lists.

    Parameters:
    - animals: list of str, names of the animals.
    - block_lists: list of lists of int, block numbers corresponding to each animal.
    - experiment_path: pathlib.Path, path to the experiment directory.
    - bad_blocks: list of int, blocks to exclude. Default is an empty list.

    Returns:
    - block_collection: list of BlockSync objects for all specified blocks.
    - block_dict: dictionary where keys are block numbers as strings and values are BlockSync objects.
    """
    import UtilityFunctions_newOE as uf

    if bad_blocks is None:
        bad_blocks = []

    block_collection = []
    block_dict = {}

    for animal, blocks in zip(animals, block_lists):
        # Generate blocks for the current animal
        current_blocks = uf.block_generator(
            block_numbers=blocks,
            experiment_path=experiment_path,
            animal=animal,
            bad_blocks=bad_blocks
        )
        # Add to collection and dictionary
        block_collection.extend(current_blocks)
        for b in current_blocks:
            block_dict[f"{animal}_block_{b.block_num}"] = b

    return block_collection, block_dict


animals = ['PV_62', 'PV_126', 'PV_57']
#block_lists = [[23, 24, 26, 38], [7, 8, 9, 10, 11, 12], [7, 8, 9, 11, 12, 13]]
block_lists = [[],[13],[]]
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
bad_blocks = [0]  # Example of bad blocks

block_collection, block_dict = create_block_collections(
    animals=animals,
    block_lists=block_lists,
    experiment_path=experiment_path,
    bad_blocks=bad_blocks
)

instantiated block number 013 at Path: Z:\Nimrod\experiments\PV_126\2024_08_13\block_013, new OE version
Found the sample rate for block 013 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 013
got it!


In [127]:
for block in block_collection:

    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)
    block.handle_eye_videos()
    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    load_eye_data_2d_w_rotation_matrix(block)

    #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_3d_corr_verified.csv')
    #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_3d_corr_verified.csv')

running parse_open_ephys_events...
block 013 has a parsed events file, reading...
Getting eye brightness values for block 013...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder
handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_013\\eye_videos\\LE\\126_hunter9_640x480_60hz_experiment_1_recording_0\\126_hunter9.h264', 'Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_013\\eye_videos\\RE\\126_hunter9_640x480_60hz_experiment_1_recording_0\\126_hunter9.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_013\\eye_videos\\LE\\126_hunter9_640x480_60hz_experiment_1_recording_0\\126_hunter9_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_126\\2024_08_13\\block_013\\eye_videos\\RE\\126_hunter9_640x480_60hz_experiment_1_recording_0\\126_hunter9.mp4']
The file Z:\Nimrod\experiments\PV_126\2024_08

In [128]:
block_dict.keys()

dict_keys(['PV_126_block_013'])

In [133]:
# set up a block
block = block_dict['PV_126_block_013']
#block.load_best_reference(r'Z:\Nimrod\experiments\cross_animals_data\kerr_reference_all_animals_current_25_05_12.csv')

In [121]:
r_ref = tuple([int(block.kerr_ref_r_x), int(block.kerr_ref_r_y)])
l_ref = tuple([int(block.kerr_ref_l_x), int(block.kerr_ref_l_y)])

In [134]:
# use the interactive tool to get a properly aligned data on a native frame (this is y-flipped after plotting and maintains the y-positive = up convention
# save before exiting the tool!!
interactive_eye_data_corrector_synced(block, eye='left', ref_point_xy=None)

Left eye data saved.


In [135]:
# run for right eye
interactive_eye_data_corrector_synced(block, eye='right', ref_point_xy=None)

In [136]:
# Use this tool to verify correct tearducts-based correction, does not automatically updates the rotation matrix
interactive_eye_rotation_checker(block, eye='right')

In [92]:
# and the left eye
interactive_eye_rotation_checker(block, eye='left')

In [137]:
# if required, uncomment to negate rotation matrices
negate_eye_rotation(block, 'left')
negate_eye_rotation(block, 'right')

Left rotation matrix negated; angle set to 17.06°
Right rotation matrix negated; angle set to 347.91°


In [138]:
# finally, export data by overwriting:
export_corrected_eye_data(block)

Exported corrected eye data and rotation params to Z:\Nimrod\experiments\PV_126\2024_08_13\block_013\analysis


In [40]:
def interactive_eye_data_corrector_synced_with_matrix(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor with Play/Pause, correction,
    Save, Flip-Dot, Skip, AND:
      • Matrix Rotate Frame: toggle warping the frame by the saved rotation matrix
      • Matrix Rotate Data:  apply the saved rotation matrix to the overlay data

    Parameters
    ----------
    block : BlockSync
        Must have left_/right_eye_data, left_/right_rotation_matrix, etc.
    eye : 'left' or 'right'
    ref_point_xy : (x,y) or None
        Optional blue dot coordinate to draw on every frame.
    """
    import cv2, numpy as np, pandas as pd

    # Select eye‐specific attributes
    if eye.lower()=='left':
        df_orig   = block.left_eye_data.copy()
        rot_mat   = np.array(block.left_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.left_rotation_angle)
        video     = block.le_videos[0]
    else:
        df_orig   = block.right_eye_data.copy()
        rot_mat   = np.array(block.right_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.right_rotation_angle)
        video     = block.re_videos[0]

    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip = int(fps*60)

    # DataFrame and frame index column
    df_current = df_orig.copy()
    frame_col  = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # Buttons
    buttons = {
        'Play':               ((10,   10), (190,  60)),
        'Pause':              ((10,   70), (190, 120)),
        'Un-rotate':          ((10,  130), (190, 180)),
        'X-flip':             ((10,  190), (190, 240)),
        'Re-rotate':          ((10,  250), (190, 300)),
        'Phi+90':             ((10,  310), (190, 360)),
        'FlipX-only':         ((10,  370), (190, 420)),
        'Flip Dot':           ((10,  430), (190, 480)),
        'Bwd':                ((10,  490), (190, 540)),
        'Fwd':                ((10,  550), (190, 600)),
        'Matrix Rotate Frame':((10,  610), (190, 660)),
        'Matrix Rotate Data': ((10,  670), (190, 720)),
        'Save':               ((10,  730), (190, 780)),
        'Quit':               ((10,  790), (190, 840)),
    }
    ctrl_h = 850; ctrl_w = 200

    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            cv2.rectangle(img, (x1,y1), (x2,y2), (50,50,50), -1)
            cv2.rectangle(img, (x1,y1), (x2,y2), (200,200,200), 2)
            cv2.putText(img, name, (x1+5, y1+35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
        return img

    controls_img = draw_controls()
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)

    # State flags
    running        = True
    playing        = False
    current_ref    = ref_point_xy
    last_frame     = None
    warp_frame     = False

    # Mouse callback
    def on_mouse(event, x, y, flags, param):
        nonlocal df_current, running, playing, current_ref, last_frame, warp_frame
        if event!=cv2.EVENT_LBUTTONDOWN:
            return
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            if x1<=x<=x2 and y1<=y<=y2:
                if   name=='Play':               playing = True
                elif name=='Pause':              playing = False
                elif name=='Un-rotate':          df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name=='X-flip':             df_current = horizontal_flip_eye_data( df_current, W)
                elif name=='Re-rotate':          df_current = apply_rotation( df_current, rot_mat, rot_angle)
                elif name=='Phi+90':             df_current = rotate_phi_only(df_current)
                elif name=='FlipX-only':         df_current = flip_x_only(df_current, W)
                elif name=='Flip Dot' and current_ref is not None:
                    x0,y0 = current_ref
                    current_ref = (W-x0, y0)
                elif name=='Bwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, max(idx-skip,0))
                    last_frame=None
                elif name=='Fwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, min(idx+skip, total_frames-1))
                    last_frame=None
                elif name=='Matrix Rotate Frame':
                    warp_frame = not warp_frame
                elif name=='Matrix Rotate Data':
                    df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name=='Save':
                    if eye.lower()=='left':
                        block.left_eye_data = df_current.copy()
                    else:
                        block.right_eye_data = df_current.copy()
                    print(f"{eye} eye data saved.")
                elif name=='Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # Main loop
    while running:
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret: break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        # Optionally warp the frame
        display_frame = frame
        if warp_frame:
            display_frame = cv2.warpAffine(display_frame, rot_mat,
                                           (W, display_frame.shape[0]),
                                           flags=cv2.INTER_LINEAR,
                                           borderMode=cv2.BORDER_CONSTANT,
                                           borderValue=(0,0,0))

        # Sync current frame index
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
        idx = max(idx,0)

        # Annotate
        annotated = display_frame.copy()
        # reference dot
        if current_ref is not None:
            cv2.circle(annotated, current_ref, 5, (255,0,0), -1)
        # ellipse
        mask = df_current[frame_col]==idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx,cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                x,y = int(round(cx)), int(round(cy))
                w,h = int(row['width']), int(row['height'])
                phi = float(row['phi'])
                cv2.ellipse(annotated, (x,y), (w,h), phi, 0,360, (0,255,0),2)

        # final vertical flip for display
        disp = cv2.flip(annotated, 0)
        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        if cv2.waitKey(30)&0xFF==27:
            break

    cap.release()
    cv2.destroyAllWindows()


In [43]:
interactive_eye_data_corrector_synced_with_matrix(block,'right')

# These could be useful someday - here for reference:

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_extreme_frames(video_path, eye_data, figsize=(10,8), cmap_frame=None):
    """
    Plot the 4 most extreme pupil detections on a 2×2 grid:
      - min center_x (leftmost)
      - max center_x (rightmost)
      - min center_y (topmost)
      - max center_y (bottommost)

    Parameters
    ----------
    video_path : str
        Path to your video file.
    eye_data : pandas.DataFrame
        Must have columns ['eye_frame', 'center_x', 'center_y',
        'phi', 'major_ax', 'minor_ax'].
    figsize : tuple
        Matplotlib figure size.
    cmap_frame : str or None
        Matplotlib cmap for the frames (e.g. 'gray') or None for RGB.
    """
    # 1. Clean out any rows with NaNs in the required columns
    req = ['eye_frame','center_x','center_y','phi','major_ax','minor_ax']
    df = eye_data.dropna(subset=req)

    if df.empty:
        raise ValueError("No valid rows after dropping NaNs.")

    # 2. Pick the four extremes
    picks = {
        'leftmost':  df.loc[df['center_x'].idxmin()],
        'rightmost': df.loc[df['center_x'].idxmax()],
        'topmost':   df.loc[df['center_y'].idxmin()],
        'bottommost':df.loc[df['center_y'].idxmax()],
    }

    # 3. Grab frames
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video {video_path!r}")

    # store (image, row) for each
    samples = {}
    for name, row in picks.items():
        frm = int(row.eye_frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frm)
        ret, img = cap.read()
        if not ret:
            raise IOError(f"Failed to read frame {frm} from {video_path!r}")
        # convert BGR→RGB for plotting
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        samples[name] = (img, row)

    cap.release()

    # 4. Plot them
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    axes = axes.flatten()

    for ax, (name, (img, row)) in zip(axes, samples.items()):
        # show frame
        if cmap_frame:
            ax.imshow(img[...,0], cmap=cmap_frame)  # single‐channel
        else:
            ax.imshow(img)

        # overlay ellipse
        # matplotlib Ellipse wants width/height = full diameters.
        w, h = 2*row.major_ax, 2*row.minor_ax
        e = Ellipse(
            (row.center_x, row.center_y),
            width=w, height=h,
            angle=row.phi,
            edgecolor='lime', facecolor='none', lw=2
        )
        ax.add_patch(e)

        # annotate text
        txt = (
            f"{name}\n"
            f"frame={int(row.eye_frame)}\n"
            f"x={row.center_x:.1f}, y={row.center_y:.1f}\n"
            f"φ={row.phi:.1f}°"
        )
        ax.text(
            0.02, 0.95, txt,
            transform=ax.transAxes,
            va='top', ha='left',
            color='white', fontsize=9,
            bbox=dict(facecolor='black', alpha=0.5, pad=3)
        )

        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(name, fontsize=10)

    plt.tight_layout()
    plt.show()

block = block_dict['PV_126_block_011']
plot_extreme_frames(block.le_videos[0],block.left_eye_data)

In [53]:
def interactive_eye_data_corrector_synced_with_vector(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor displaying an OpenCV window
    for the frame+data and a second OpenCV window showing the 3D vector (k_phi, k_theta).
    """
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # 1) select eye‐specific data
    if eye.lower() == 'left':
        df_orig = block.left_eye_data.copy()
        video   = block.le_videos[0]
    else:
        df_orig = block.right_eye_data.copy()
        video   = block.re_videos[0]

    # open video
    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")

    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip = int(fps * 60)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # DataFrame & frame index column
    df_current = df_orig.copy()
    frame_col  = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # --- Matplotlib figure setup (offscreen) ---
    plt.ioff()
    fig = plt.figure(figsize=(3, 3), dpi=100)
    ax3 = fig.add_subplot(111, projection='3d')
    ax3.set_xlim(-1, 1); ax3.set_ylim(-1, 1); ax3.set_zlim(-1, 1)
    ax3.set_xlabel('X'); ax3.set_ylabel('Y'); ax3.set_zlabel('Z')

    # --- OpenCV controls setup ---
    buttons = {
        'Play':        ((10,  10),(180,  60)),
        'Pause':       ((10,  70),(180, 120)),
        'Un-rotate':   ((10, 130),(180, 180)),
        'X-flip':      ((10, 190),(180, 240)),
        'Re-rotate':   ((10, 250),(180, 300)),
        'Phi+90':      ((10, 310),(180, 360)),
        'FlipX-only':  ((10, 370),(180, 420)),
        'Flip Dot':    ((10, 430),(180, 480)),
        'Bwd':         ((10, 490),(180, 540)),
        'Fwd':         ((10, 550),(180, 600)),
        'Save':        ((10, 610),(180, 660)),
        'Quit':        ((10, 670),(180, 720)),
    }
    ctrl_h, ctrl_w = 760, 200

    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            cv2.rectangle(img, (x1, y1), (x2, y2), (50, 50, 50), -1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (200, 200, 200), 2)
            cv2.putText(img, name, (x1 + 5, y1 + 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2, cv2.LINE_AA)
        return img

    controls_img = draw_controls()
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Frame',    cv2.WINDOW_NORMAL)
    cv2.namedWindow('Vector',   cv2.WINDOW_NORMAL)

    # State
    running     = True
    playing     = False
    current_ref = ref_point_xy
    last_frame  = None

    def on_mouse(event, x, y, flags, param):
        nonlocal df_current, running, playing, current_ref, last_frame
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        for name, ((x1, y1), (x2, y2)) in buttons.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                if   name == 'Play':       playing = True
                elif name == 'Pause':      playing = False
                elif name == 'Un-rotate':  df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name == 'X-flip':     df_current = horizontal_flip_eye_data(df_current, W)
                elif name == 'Re-rotate':  df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name == 'Phi+90':     df_current = rotate_phi_only(df_current)
                elif name == 'FlipX-only': df_current = flip_x_only(df_current, W)
                elif name == 'Flip Dot' and current_ref is not None:
                    x0, y0 = current_ref
                    current_ref = (W - x0, y0)
                elif name == 'Bwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, max(idx - skip, 0)); last_frame = None
                elif name == 'Fwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, min(idx + skip, total_frames - 1)); last_frame = None
                elif name == 'Save':
                    if eye.lower() == 'left':
                        block.left_eye_data  = df_current.copy()
                    else:
                        block.right_eye_data = df_current.copy()
                elif name == 'Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # Main loop
    while running:
        # get frame
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret:
                break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        # current frame index
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        idx = max(idx, 0)

        # 1) Annotate frame + data
        annotated = frame.copy()
        if current_ref is not None:
            cv2.circle(annotated, current_ref, 5, (255, 0, 0), -1)
        mask = df_current[frame_col] == idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx, cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                cv2.ellipse(annotated,
                            (int(round(cx)), int(round(cy))),
                            (int(row['width']), int(row['height'])),
                            float(row['phi']), 0, 360, (0, 255, 0), 2)
        disp = cv2.flip(annotated, 0)
        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        # 2) Build vector window image via Matplotlib canvas
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = np.deg2rad(row['k_phi'])
            kt = np.deg2rad(row['k_theta'])
            x = np.cos(kt) * np.cos(kp)
            y = np.cos(kt) * np.sin(kp)
            z = np.sin(kt)

            # redraw 3D arrow
            ax3.cla()
            ax3.set_xlim(-1,1); ax3.set_ylim(-1,1); ax3.set_zlim(-1,1)
            ax3.set_xlabel('X'); ax3.set_ylabel('Y'); ax3.set_zlabel('Z')
            ax3.quiver(0,0,0, x,y,z, length=1, normalize=True, color='blue')

            fig.canvas.draw()
            # grab canvas as RGB array
            w, h = fig.canvas.get_width_height()
            buf = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            buf = buf.reshape((h, w, 3))
            vec_img = cv2.cvtColor(buf, cv2.COLOR_RGB2BGR)
            cv2.imshow('Vector', vec_img)

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    plt.close(fig)



In [80]:
# with corrected column
def interactive_eye_data_corrector_synced_with_vector(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor with two OpenCV windows:
      • 'Frame'    : video with ellipse/data overlay
      • 'Controls' : play/pause, corrections, save, etc.
      • 'Vector'   : 3D vector of (k_phi, k_theta) plus a red reference at (0,0)
      • 'VectorCorr': 3D vector of (k_phi_corr, k_theta_corr) plus red reference

    Requires in scope:
      apply_inverse_rotation, horizontal_flip_eye_data,
      apply_rotation, rotate_phi_only, flip_x_only
    """
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # --- 1) Select data & video path ---
    if eye.lower() == 'left':
        df_orig = block.left_eye_data.copy()
        video   = block.le_videos[0]
    else:
        df_orig = block.right_eye_data.copy()
        video   = block.re_videos[0]

    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")

    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip = int(fps * 60)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    df_current = df_orig.copy()
    frame_col  = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # --- 2) Off‐screen Matplotlib setup for two 3D plots ---
    plt.ioff()
    # Original angles
    fig1 = plt.figure(figsize=(3, 3), dpi=100)
    ax1  = fig1.add_subplot(111, projection='3d')
    ax1.set_xlim(-1,1); ax1.set_ylim(-1,1); ax1.set_zlim(-1,1)
    ax1.set_title('k\_phi / k\_theta')
    # Corrected angles
    fig2 = plt.figure(figsize=(3, 3), dpi=100)
    ax2  = fig2.add_subplot(111, projection='3d')
    ax2.set_xlim(-1,1); ax2.set_ylim(-1,1); ax2.set_zlim(-1,1)
    ax2.set_title('k\_phi\_corr / k\_theta\_corr')

    # --- 3) OpenCV UI windows ---
    cv2.namedWindow('Frame',    cv2.WINDOW_NORMAL)
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Vector',   cv2.WINDOW_NORMAL)
    cv2.namedWindow('VectorCorr', cv2.WINDOW_NORMAL)

    # Build controls layout (buttons as before)...
    # [ same buttons dict and draw_controls() as previous version ]
    buttons = {
        'Play':        ((10, 10),(180, 60)),
        'Pause':       ((10, 70),(180,120)),
        'Un-rotate':   ((10,130),(180,180)),
        'X-flip':      ((10,190),(180,240)),
        'Re-rotate':   ((10,250),(180,300)),
        'Phi+90':      ((10,310),(180,360)),
        'FlipX-only':  ((10,370),(180,420)),
        'Flip Dot':    ((10,430),(180,480)),
        'Bwd':         ((10,490),(180,540)),
        'Fwd':         ((10,550),(180,600)),
        'Save':        ((10,610),(180,660)),
        'Quit':        ((10,670),(180,720)),
    }
    ctrl_h, ctrl_w = 740, 200
    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            cv2.rectangle(img, (x1,y1), (x2,y2), (50,50,50), -1)
            cv2.rectangle(img, (x1,y1), (x2,y2), (200,200,200), 2)
            cv2.putText(img, name, (x1+5, y1+35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
        return img
    controls_img = draw_controls()

    # --- 4) Interaction state & callbacks ---
    running     = True
    playing     = False
    current_ref = ref_point_xy
    last_frame  = None

    def on_mouse(event, x, y, flags, param):
        nonlocal df_current, running, playing, current_ref, last_frame
        if event != cv2.EVENT_LBUTTONDOWN: return
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            if x1<=x<=x2 and y1<=y<=y2:
                if   name=='Play':        playing = True
                elif name=='Pause':       playing = False
                elif name=='Un-rotate':   df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name=='X-flip':      df_current = horizontal_flip_eye_data(df_current, W)
                elif name=='Re-rotate':   df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name=='Phi+90':      df_current = rotate_phi_only(df_current)
                elif name=='FlipX-only':  df_current = flip_x_only(df_current, W)
                elif name=='Flip Dot' and current_ref is not None:
                    x0,y0 = current_ref
                    current_ref = (W - x0, y0)
                elif name=='Bwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, max(idx-skip,0)); last_frame=None
                elif name=='Fwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, min(idx+skip,total_frames-1)); last_frame=None
                elif name=='Save':
                    if eye.lower()=='left':
                        block.left_eye_data = df_current.copy()
                    else:
                        block.right_eye_data = df_current.copy()
                elif name=='Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # --- 5) Main loop ---
    while running:
        # frame fetch
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret: break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        idx = max(idx, 0)

        # annotate frame + ellipse
        annotated = frame.copy()
        if current_ref is not None:
            cv2.circle(annotated, current_ref, 5, (255,0,0), -1)
        mask = df_current[frame_col] == idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx, cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                cv2.ellipse(
                    annotated,
                    (int(round(cx)), int(round(cy))),
                    (int(row['width']), int(row['height'])),
                    float(row['phi']), 0,360, (0,255,0), 2
                )

        disp = cv2.flip(annotated, 0)
        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        # --- original k_phi / k_theta vector ---
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = np.deg2rad(row['k_phi'])
            kt = np.deg2rad(row['k_theta'])
            x = np.cos(kt)*np.cos(kp)
            y = np.cos(kt)*np.sin(kp)
            z = np.sin(kt)

            ax1.cla()
            ax1.set_xlim(-1,1); ax1.set_ylim(-1,1); ax1.set_zlim(-1,1)
            # red reference vector at (0,0)
            ax1.quiver(0,0,0, 1,0,0, length=0.5, normalize=True, color='red')
            # dynamic vector
            ax1.quiver(0,0,0, x,y,z, length=1, normalize=True, color='blue')
            fig1.canvas.draw()
            buf = np.frombuffer(fig1.canvas.tostring_rgb(), dtype=np.uint8)
            h, w = fig1.canvas.get_width_height()
            buf = buf.reshape((h, w, 3))
            cv2.imshow('Vector', cv2.cvtColor(buf, cv2.COLOR_RGB2BGR))

        # --- corrected k_phi_corr / k_theta_corr vector ---
        if mask.any() and 'corr_phi' in df_current.columns and 'corr_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp2 = np.deg2rad(row['corr_phi'])
            kt2 = np.deg2rad(row['corr_theta'])
            x2 = np.cos(kt2)*np.cos(kp2)
            y2 = np.cos(kt2)*np.sin(kp2)
            z2 = np.sin(kt2)

            ax2.cla()
            ax2.set_xlim(-1,1); ax2.set_ylim(-1,1); ax2.set_zlim(-1,1)
            ax2.quiver(0,0,0, 1,0,0, length=0.5, normalize=True, color='red')
            ax2.quiver(0,0,0, x2,y2,z2, length=1, normalize=True, color='green')
            fig2.canvas.draw()
            buf2 = np.frombuffer(fig2.canvas.tostring_rgb(), dtype=np.uint8)
            h2, w2 = fig2.canvas.get_width_height()
            buf2 = buf2.reshape((h2, w2, 3))
            cv2.imshow('VectorCorr', cv2.cvtColor(buf2, cv2.COLOR_RGB2BGR))

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    plt.close(fig1)
    plt.close(fig2)


In [66]:
interactive_eye_data_corrector_synced_with_vector(block, eye='right', ref_point_xy=r_ref)

In [82]:
interactive_eye_data_corrector_synced_with_vector(block, eye='left', ref_point_xy=l_ref)

In [75]:
def interactive_eye_data_corrector_synced_with_vector(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor with:
      • 'Frame'    : video + ellipse + ref dot + phi/theta text overlay
      • 'Controls' : playback & transform buttons
      • 'Vector'   : 3D vector of (k_phi, k_theta) with red 0° ref
      • 'VectorCorr': 3D vector of (corr_phi, corr_theta) with red 0° ref

    Parameters
    ----------
    block : BlockSync
        Must have left_/right_eye_data including columns
        ['center_x','center_y','width','height','phi','k_phi','k_theta',
         'corr_phi','corr_theta'], and rotation attributes.
    eye : 'left' or 'right'
    ref_point_xy : tuple[int,int] or None
        Optional reference point to draw as blue dot.
    """
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # Pick data & video
    if eye.lower() == 'left':
        df_current = block.left_eye_data.copy()
        rot_mat = np.array(block.left_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.left_rotation_angle)
        video = block.le_videos[0]
    else:
        df_current = block.right_eye_data.copy()
        rot_mat = np.array(block.right_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.right_rotation_angle)
        video = block.re_videos[0]

    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip = int(fps * 60)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_col = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # Offscreen Matplotlib figures
    plt.ioff()
    fig1 = plt.figure(figsize=(3,3), dpi=100)
    ax1 = fig1.add_subplot(111, projection='3d')
    fig2 = plt.figure(figsize=(3,3), dpi=100)
    ax2 = fig2.add_subplot(111, projection='3d')

    # OpenCV windows
    cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Vector', cv2.WINDOW_NORMAL)
    cv2.namedWindow('VectorCorr', cv2.WINDOW_NORMAL)

    # Buttons
    buttons = {
        'Play':((10,10),(180,60)), 'Pause':((10,70),(180,120)),
        'Un-rotate':((10,130),(180,180)), 'X-flip':((10,190),(180,240)),
        'Re-rotate':((10,250),(180,300)), 'Phi+90':((10,310),(180,360)),
        'FlipX-only':((10,370),(180,420)), 'Flip Dot':((10,430),(180,480)),
        'Bwd':((10,490),(180,540)), 'Fwd':((10,550),(180,600)),
        'Save':((10,610),(180,660)), 'Quit':((10,670),(180,720)),
    }
    ctrl_h, ctrl_w = 740, 200
    def draw_controls():
        img = np.zeros((ctrl_h,ctrl_w,3), np.uint8)
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            cv2.rectangle(img,(x1,y1),(x2,y2),(50,50,50),-1)
            cv2.rectangle(img,(x1,y1),(x2,y2),(200,200,200),2)
            cv2.putText(img,name,(x1+5,y1+35),cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,(200,200,200),2,cv2.LINE_AA)
        return img
    controls_img = draw_controls()

    running = True
    playing = False
    current_ref = ref_point_xy
    last_frame = None

    def on_mouse(evt,x,y,flags,param):
        nonlocal df_current, running, playing, current_ref, last_frame
        if evt!=cv2.EVENT_LBUTTONDOWN: return
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            if x1<=x<=x2 and y1<=y<=y2:
                if name=='Play': playing=True
                elif name=='Pause': playing=False
                elif name=='Un-rotate':
                    df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name=='X-flip':
                    df_current = horizontal_flip_eye_data(df_current, W)
                elif name=='Re-rotate':
                    df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name=='Phi+90':
                    df_current = rotate_phi_only(df_current)
                elif name=='FlipX-only':
                    df_current = flip_x_only(df_current, W)
                elif name=='Flip Dot' and current_ref is not None:
                    x0,y0=current_ref; current_ref=(W-x0,y0)
                elif name=='Bwd':
                    idx=int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
                    cap.set(cv2.CAP_PROP_POS_FRAMES,max(idx-skip,0)); last_frame=None
                elif name=='Fwd':
                    idx=int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
                    cap.set(cv2.CAP_PROP_POS_FRAMES,min(idx+skip,total_frames-1)); last_frame=None
                elif name=='Save':
                    if eye.lower()=='left': block.left_eye_data=df_current.copy()
                    else: block.right_eye_data=df_current.copy()
                elif name=='Quit': running=False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    while running:
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret: break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1
        idx = max(idx,0)

        # Frame annotation
        ann = frame.copy()
        if current_ref is not None:
            cv2.circle(ann, current_ref, 5, (255,0,0), -1)
        mask = df_current[frame_col]==idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx,cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                cv2.ellipse(ann,(int(round(cx)),int(round(cy))),
                            (int(row['width']),int(row['height'])),
                            float(row['phi']),0,360,(0,255,0),2)
        disp = cv2.flip(ann,0)

        # Overlay plain text
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = row['k_phi']; kt = row['k_theta']
            txt = f"phi={kp:.1f}, theta={kt:.1f}"
            cv2.putText(disp, txt, (10, disp.shape[0]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255),2,cv2.LINE_AA)

        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        # Vector plot for (k_phi, k_theta)
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = np.deg2rad(row['k_phi']); kt = np.deg2rad(row['k_theta'])
            x,y,z = np.cos(kt)*np.cos(kp), np.cos(kt)*np.sin(kp), np.sin(kt)
            ax1.cla()
            ax1.set_xlim(-1,1); ax1.set_ylim(-1,1); ax1.set_zlim(-1,1)
            ax1.quiver(0,0,0,1,0,0,length=0.5,normalize=True,color='red')
            ax1.quiver(0,0,0,x,y,z,length=1,normalize=True,color='blue')
            fig1.canvas.draw()
            buf = np.frombuffer(fig1.canvas.tostring_rgb(),np.uint8)
            h,w = fig1.canvas.get_width_height()
            vec = buf.reshape((h,w,3))
            cv2.imshow('Vector', cv2.cvtColor(vec, cv2.COLOR_RGB2BGR))

        # VectorCorr for (corr_phi, corr_theta)
        if mask.any() and 'corr_phi' in df_current.columns and 'corr_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp2 = np.deg2rad(row['corr_phi']); kt2 = np.deg2rad(row['corr_theta'])
            x2,y2,z2 = np.cos(kt2)*np.cos(kp2), np.cos(kt2)*np.sin(kp2), np.sin(kt2)
            ax2.cla()
            ax2.set_xlim(-1,1); ax2.set_ylim(-1,1); ax2.set_zlim(-1,1)
            ax2.quiver(0,0,0,1,0,0,length=0.5,normalize=True,color='red')
            ax2.quiver(0,0,0,x2,y2,z2,length=1,normalize=True,color='green')
            fig2.canvas.draw()
            buf2 = np.frombuffer(fig2.canvas.tostring_rgb(),np.uint8)
            h2,w2 = fig2.canvas.get_width_height()
            vec2 = buf2.reshape((h2,w2,3))
            cv2.imshow('VectorCorr', cv2.cvtColor(vec2, cv2.COLOR_RGB2BGR))

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    plt.close(fig1); plt.close(fig2)



In [81]:
def interactive_eye_data_corrector_synced_with_vector(block, eye, ref_point_xy=None):
    """
    Interactive synchronized video + ellipse editor with:
      • Frame window: raw video + ellipse + ref dot + k_phi/k_theta text overlay
      • Controls window: buttons for playback & transforms
      • Vector window: 3D vector of (k_phi, k_theta) with red 0° reference
      • VectorCorr window: 3D vector of (k_phi_corr, k_theta_corr) with red 0° reference

    Parameters
    ----------
    block : BlockSync
        Must have left_/right_eye_data with columns ['center_x','center_y','width','height','phi',
        'k_phi','k_theta','k_phi_corr','k_theta_corr'], plus rotation attributes.
    eye : 'left' or 'right'
    ref_point_xy : tuple[int,int] or None
        Optional reference point to draw as a blue dot.
    """
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # 1) Select data & video
    if eye.lower() == 'left':
        df_orig   = block.left_eye_data.copy()
        rot_mat   = np.array(block.left_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.left_rotation_angle)
        video     = block.le_videos[0]
    else:
        df_orig   = block.right_eye_data.copy()
        rot_mat   = np.array(block.right_rotation_matrix, dtype=np.float32)
        rot_angle = float(block.right_rotation_angle)
        video     = block.re_videos[0]

    cap = cv2.VideoCapture(str(video))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {eye} video: {video}")

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    skip = int(fps * 60)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    df_current = df_orig.copy()
    frame_col  = 'eye_frame' if 'eye_frame' in df_current.columns else 'frame'

    # 2) Offscreen Matplotlib for two 3D plots
    plt.ioff()
    fig1 = plt.figure(figsize=(3, 3), dpi=100)
    ax1  = fig1.add_subplot(111, projection='3d')
    fig2 = plt.figure(figsize=(3, 3), dpi=100)
    ax2  = fig2.add_subplot(111, projection='3d')

    # 3) OpenCV windows
    cv2.namedWindow('Frame',    cv2.WINDOW_NORMAL)
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Vector',   cv2.WINDOW_NORMAL)
    cv2.namedWindow('VectorCorr', cv2.WINDOW_NORMAL)

    # Buttons layout
    buttons = {
        'Play':        ((10,  10), (180,  60)),
        'Pause':       ((10,  70), (180, 120)),
        'Un-rotate':   ((10, 130), (180, 180)),
        'X-flip':      ((10, 190), (180, 240)),
        'Re-rotate':   ((10, 250), (180, 300)),
        'Phi+90':      ((10, 310), (180, 360)),
        'FlipX-only':  ((10, 370), (180, 420)),
        'Flip Dot':    ((10, 430), (180, 480)),
        'Bwd':         ((10, 490), (180, 540)),
        'Fwd':         ((10, 550), (180, 600)),
        'Save':        ((10, 610), (180, 660)),
        'Quit':        ((10, 670), (180, 720)),
    }
    ctrl_h, ctrl_w = 740, 200

    def draw_controls():
        img = np.zeros((ctrl_h, ctrl_w, 3), dtype=np.uint8)
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            cv2.rectangle(img, (x1,y1), (x2,y2), (50,50,50), -1)
            cv2.rectangle(img, (x1,y1), (x2,y2), (200,200,200), 2)
            cv2.putText(img, name, (x1+5, y1+35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
        return img

    controls_img = draw_controls()

    # 4) Interaction state
    running     = True
    playing     = False
    current_ref = ref_point_xy
    last_frame  = None

    # 5) Mouse callback
    def on_mouse(event, x, y, flags, param):
        nonlocal df_current, running, playing, current_ref, last_frame
        if event != cv2.EVENT_LBUTTONDOWN:
            return
        for name, ((x1,y1),(x2,y2)) in buttons.items():
            if x1<=x<=x2 and y1<=y<=y2:
                if   name == 'Play':
                    playing = True
                elif name == 'Pause':
                    playing = False
                elif name == 'Un-rotate':
                    df_current = apply_inverse_rotation(df_current, rot_mat, rot_angle)
                elif name == 'X-flip':
                    df_current = horizontal_flip_eye_data(df_current, W)
                elif name == 'Re-rotate':
                    df_current = apply_rotation(df_current, rot_mat, rot_angle)
                elif name == 'Phi+90':
                    df_current = rotate_phi_only(df_current)
                elif name == 'FlipX-only':
                    df_current = flip_x_only(df_current, W)
                elif name == 'Flip Dot' and current_ref is not None:
                    x0, y0 = current_ref
                    current_ref = (W - x0, y0)
                elif name == 'Bwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, max(idx - skip, 0))
                    last_frame = None
                elif name == 'Fwd':
                    idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                    cap.set(cv2.CAP_PROP_POS_FRAMES, min(idx + skip, total_frames - 1))
                    last_frame = None
                elif name == 'Save':
                    if eye.lower() == 'left':
                        block.left_eye_data  = df_current.copy()
                    else:
                        block.right_eye_data = df_current.copy()
                elif name == 'Quit':
                    running = False
                break

    cv2.setMouseCallback('Controls', on_mouse)

    # 6) Main loop
    while running:
        # Read or reuse frame
        if playing or last_frame is None:
            ret, frame = cap.read()
            if not ret:
                break
            last_frame = frame.copy()
        else:
            frame = last_frame.copy()

        # Sync index
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        idx = max(idx, 0)

        # Annotate frame
        annotated = frame.copy()
        if current_ref is not None:
            cv2.circle(annotated, current_ref, 5, (255,0,0), -1)
        mask = df_current[frame_col] == idx
        if mask.any():
            row = df_current[mask].iloc[0]
            cx, cy = row['center_x'], row['center_y']
            if not (pd.isna(cx) or pd.isna(cy)):
                cv2.ellipse(
                    annotated,
                    (int(round(cx)), int(round(cy))),
                    (int(row['width']), int(row['height'])),
                    float(row['phi']), 0, 360,
                    (0,255,0), 2
                )

        # Flip for display
        disp = cv2.flip(annotated, 0)

        # Overlay k_phi / k_theta text
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = row['k_phi']
            kt = row['k_theta']
            txt = f"k_phi: {kp:.1f}°, k_theta: {kt:.1f}°"
            cv2.putText(
                disp, txt,
                (10, disp.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6, (255,255,255), 2, cv2.LINE_AA
            )

        cv2.imshow('Frame', disp)
        cv2.imshow('Controls', controls_img)

        # Draw 3D vector for (k_phi, k_theta)
        if mask.any() and 'k_phi' in df_current.columns and 'k_theta' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp = np.deg2rad(row['k_phi'])
            kt = np.deg2rad(row['k_theta'])
            x = np.cos(kt) * np.cos(kp)
            y = np.cos(kt) * np.sin(kp)
            z = np.sin(kt)

            ax1.cla()
            ax1.view_init(elev=0, azim=0)
            ax1.set_xlim(-1,1); ax1.set_ylim(-1,1); ax1.set_zlim(-1,1)
            # red 0° ref
            ax1.quiver(0,0,0, 1,0,0, length=0.5, normalize=True, color='red')
            ax1.quiver(0,0,0, x,y,z, length=1, normalize=True, color='blue')
            fig1.canvas.draw()
            buf = np.frombuffer(fig1.canvas.tostring_rgb(), dtype=np.uint8)
            h, w = fig1.canvas.get_width_height()
            buf = buf.reshape((h, w, 3))
            cv2.imshow('Vector', cv2.cvtColor(buf, cv2.COLOR_RGB2BGR))

        # Draw 3D vector for (k_phi_corr, k_theta_corr)
        if mask.any() and 'k_phi_corr' in df_current.columns and 'k_theta_corr' in df_current.columns:
            row = df_current[mask].iloc[0]
            kp2 = np.deg2rad(row['k_phi_corr'])
            kt2 = np.deg2rad(row['k_theta_corr'])
            x2 = np.cos(kt2) * np.cos(kp2)
            y2 = np.cos(kt2) * np.sin(kp2)
            z2 = np.sin(kt2)

            ax2.cla()
            ax2.view_init(elev=0, azim=0)
            ax2.set_xlim(-1,1); ax2.set_ylim(-1,1); ax2.set_zlim(-1,1)
            ax2.quiver(0,0,0, 1,0,0, length=0.5, normalize=True, color='red')
            ax2.quiver(0,0,0, x2,y2,z2, length=1, normalize=True, color='green')
            fig2.canvas.draw()
            buf2 = np.frombuffer(fig2.canvas.tostring_rgb(), dtype=np.uint8)
            h2, w2 = fig2.canvas.get_width_height()
            buf2 = buf2.reshape((h2, w2, 3))
            cv2.imshow('VectorCorr', cv2.cvtColor(buf2, cv2.COLOR_RGB2BGR))

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    plt.close(fig1)
    plt.close(fig2)
